# 06 - Эллиптические кривые и блокчейн-криптография

Этот notebook покрывает уроки CRYPTO-09 (Эллиптические кривые) и CRYPTO-10 (secp256k1 и Ed25519).

**Что вы изучите:**
1. Реализация ECPoint над вещественными числами
2. Реализация ECPointGF над конечными полями GF(p)
3. Визуализация кривых с matplotlib
4. secp256k1 и Ed25519 с библиотекой ecdsa
5. Скалярное умножение и ECDLP

**Используемые библиотеки:** `ecdsa`, `matplotlib`, стандартная библиотека Python

In [ ]:
# Импорты
import math
import time

print("Стандартная библиотека загружена")

---
## Часть 1: ECPoint над вещественными числами (CRYPTO-09)

Реализуем точки на эллиптической кривой y^2 = x^3 + ax + b и операцию сложения.

In [ ]:
class ECPoint:
    """Точка на эллиптической кривой y^2 = x^3 + ax + b над вещественными числами."""
    
    def __init__(self, x, y, a, b):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        # Точка на бесконечности (нейтральный элемент)
        if x is None and y is None:
            return
        # Проверяем, что точка лежит на кривой
        lhs = y ** 2
        rhs = x ** 3 + a * x + b
        assert abs(lhs - rhs) < 1e-6, \
            f"Точка ({x}, {y}) не на кривой: {lhs:.6f} != {rhs:.6f}"
    
    @property
    def is_infinity(self):
        return self.x is None
    
    def __eq__(self, other):
        if self.is_infinity and other.is_infinity:
            return True
        if self.is_infinity or other.is_infinity:
            return False
        return (abs(self.x - other.x) < 1e-10 and 
                abs(self.y - other.y) < 1e-10)
    
    def __neg__(self):
        if self.is_infinity:
            return self
        return ECPoint(self.x, -self.y, self.a, self.b)
    
    def __add__(self, other):
        # O + P = P
        if self.is_infinity:
            return other
        if other.is_infinity:
            return self
        
        # P + (-P) = O
        if abs(self.x - other.x) < 1e-10 and abs(self.y + other.y) < 1e-10:
            return ECPoint(None, None, self.a, self.b)
        
        # Вычисляем наклон
        if abs(self.x - other.x) < 1e-10:
            # Удвоение: P + P
            if abs(self.y) < 1e-10:
                return ECPoint(None, None, self.a, self.b)  # касательная вертикальна
            lam = (3 * self.x ** 2 + self.a) / (2 * self.y)
        else:
            # Сложение: P + Q
            lam = (other.y - self.y) / (other.x - self.x)
        
        x3 = lam ** 2 - self.x - other.x
        y3 = lam * (self.x - x3) - self.y
        return ECPoint(x3, y3, self.a, self.b)
    
    def __repr__(self):
        if self.is_infinity:
            return "O (бесконечность)"
        return f"({self.x:.6f}, {self.y:.6f})"

# Тест: кривая y^2 = x^3 - 3x + 5
a, b = -3, 5
O = ECPoint(None, None, a, b)  # точка на бесконечности
P = ECPoint(-1, math.sqrt(7), a, b)
Q = ECPoint(1, math.sqrt(3), a, b)

print(f"Кривая: y^2 = x^3 + ({a})x + {b}")
print(f"P = {P}")
print(f"Q = {Q}")
print(f"O = {O}")

In [ ]:
# Сложение точек
R = P + Q
print(f"P + Q = {R}")

# Удвоение
P2 = P + P
print(f"2P = P + P = {P2}")

# Свойства группы
print(f"\nP + O = {P + O}  (нейтральный элемент)")
print(f"P + (-P) = {P + (-P)}  (обратный элемент)")

# Коммутативность
PQ = P + Q
QP = Q + P
print(f"\nP + Q = {PQ}")
print(f"Q + P = {QP}")
print(f"Коммутативность: {PQ == QP}")

In [ ]:
# Скалярное умножение (double-and-add)

def scalar_multiply_real(n: int, P: ECPoint) -> ECPoint:
    """Вычисляет nP через double-and-add."""
    if n == 0:
        return ECPoint(None, None, P.a, P.b)
    if n < 0:
        return scalar_multiply_real(-n, -P)
    
    result = ECPoint(None, None, P.a, P.b)  # O
    current = P
    while n > 0:
        if n & 1:
            result = result + current
        current = current + current
        n >>= 1
    return result

# Вычисляем кратные точки
print("Скалярное умножение на кривой y^2 = x^3 - 3x + 5:")
for i in range(1, 9):
    nP = scalar_multiply_real(i, P)
    print(f"  {i}P = {nP}")

---
## Часть 2: ECPointGF над конечным полем GF(p) (CRYPTO-09)

Те же операции, но все вычисления по модулю простого числа p.

In [ ]:
class ECPointGF:
    """Точка на эллиптической кривой y^2 = x^3 + ax + b над GF(p)."""
    
    def __init__(self, x, y, a, b, p):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        self.p = p
        if x is not None:
            lhs = (y * y) % p
            rhs = (x ** 3 + a * x + b) % p
            assert lhs == rhs, f"({x}, {y}) не на кривой mod {p}: {lhs} != {rhs}"
    
    @property
    def is_infinity(self):
        return self.x is None
    
    def __eq__(self, other):
        if self.is_infinity and other.is_infinity:
            return True
        if self.is_infinity or other.is_infinity:
            return False
        return self.x == other.x and self.y == other.y
    
    def __neg__(self):
        if self.is_infinity:
            return self
        return ECPointGF(self.x, (-self.y) % self.p, self.a, self.b, self.p)
    
    def __add__(self, other):
        if self.is_infinity:
            return other
        if other.is_infinity:
            return self
        
        # P + (-P) = O
        if self.x == other.x and (self.y + other.y) % self.p == 0:
            return ECPointGF(None, None, self.a, self.b, self.p)
        
        if self.x == other.x and self.y == other.y:
            # Удвоение
            lam = (3 * self.x ** 2 + self.a) * pow(2 * self.y, -1, self.p) % self.p
        else:
            # Сложение
            lam = (other.y - self.y) * pow(other.x - self.x, -1, self.p) % self.p
        
        x3 = (lam ** 2 - self.x - other.x) % self.p
        y3 = (lam * (self.x - x3) - self.y) % self.p
        return ECPointGF(x3, y3, self.a, self.b, self.p)
    
    def __repr__(self):
        if self.is_infinity:
            return "O"
        return f"({self.x}, {self.y})"

# Тест: y^2 = x^3 + 7 mod 23 (маленький secp256k1)
p = 23
a_gf, b_gf = 0, 7

# Находим точку на кривой
# Перебираем x, ищем y такой что y^2 = x^3 + 7 (mod 23)
print(f"Кривая: y^2 = x^3 + 7 (mod {p})")
print(f"Точки на кривой:")
points = []
for x in range(p):
    rhs = (x ** 3 + b_gf) % p
    for y in range(p):
        if (y * y) % p == rhs:
            points.append((x, y))
            
print(f"  Всего точек: {len(points)} + O = {len(points) + 1}")
for pt in points[:10]:
    print(f"  ({pt[0]}, {pt[1]})")
if len(points) > 10:
    print(f"  ... и ещё {len(points) - 10} точек")

In [ ]:
# Сложение точек над GF(23)
P_gf = ECPointGF(7, 11, a_gf, b_gf, p)
Q_gf = P_gf + P_gf  # 2P

print(f"P = {P_gf}")
print(f"2P = {Q_gf}")

# Скалярное умножение над GF(p)
def scalar_multiply_gf(n: int, P: ECPointGF) -> ECPointGF:
    """nP через double-and-add над GF(p)."""
    if n == 0:
        return ECPointGF(None, None, P.a, P.b, P.p)
    if n < 0:
        return scalar_multiply_gf(-n, -P)
    result = ECPointGF(None, None, P.a, P.b, P.p)
    current = P
    while n > 0:
        if n & 1:
            result = result + current
        current = current + current
        n >>= 1
    return result

print(f"\nСкалярные кратные точки P = {P_gf}:")
for i in range(1, 30):
    nP = scalar_multiply_gf(i, P_gf)
    if nP.is_infinity:
        print(f"  {i}P = O  <-- порядок точки P = {i}")
        break
    print(f"  {i}P = {nP}")

In [ ]:
# Верификация формулы сложения на примере
P1 = ECPointGF(7, 11, 0, 7, 23)
P2 = ECPointGF(7, 11, 0, 7, 23)

# Ручное вычисление P + P (удвоение):
# lambda = (3 * 7^2 + 0) * (2 * 11)^(-1) mod 23
# lambda = 147 * 22^(-1) mod 23
# 147 mod 23 = 147 - 6*23 = 147 - 138 = 9
# 22^(-1) mod 23: 22 * x = 1 mod 23 -> x = 22 (22*22 = 484 = 21*23 + 1)
# lambda = 9 * 22 mod 23 = 198 mod 23 = 198 - 8*23 = 198 - 184 = 14
lam_manual = (9 * 22) % 23
print(f"lambda = {lam_manual}")

# x3 = lambda^2 - 2*x1 mod 23 = 14^2 - 14 mod 23 = 196 - 14 = 182 mod 23 = 182 - 7*23 = 182 - 161 = 21
x3_manual = (lam_manual**2 - 2 * 7) % 23
print(f"x3 = {x3_manual}")

# y3 = lambda * (x1 - x3) - y1 mod 23 = 14 * (7 - 21) - 11 mod 23 = 14*(-14) - 11 = -196 - 11 = -207 mod 23
y3_manual = (lam_manual * (7 - x3_manual) - 11) % 23
print(f"y3 = {y3_manual}")

# Проверяем с нашим классом
R_auto = P1 + P2
print(f"\n2P (ручное):     ({x3_manual}, {y3_manual})")
print(f"2P (автоматическое): {R_auto}")
assert R_auto.x == x3_manual and R_auto.y == y3_manual
print("Результаты совпадают!")

---
## Часть 3: Визуализация кривой над GF(p)

In [ ]:
# Визуализация точек кривой y^2 = x^3 + 7 (mod p)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for idx, p_val in enumerate([11, 23, 37]):
        xs, ys = [], []
        for x in range(p_val):
            rhs = (x ** 3 + 7) % p_val
            for y in range(p_val):
                if (y * y) % p_val == rhs:
                    xs.append(x)
                    ys.append(y)
        
        ax = axes[idx]
        ax.scatter(xs, ys, s=20, alpha=0.7, c='blue')
        ax.set_title(f'y^2 = x^3 + 7 (mod {p_val}), {len(xs)} точек')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_xlim(-1, p_val)
        ax.set_ylim(-1, p_val)
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    print("Точки выглядят 'случайными' — в отличие от гладкой кривой над R.")
    print("Но алгебраическая структура (группа) сохраняется!")

except ImportError:
    print("matplotlib не установлен. Пропускаем визуализацию.")
    print("Установите: pip install matplotlib")

---
## Часть 4: secp256k1 и Ed25519 с библиотекой ecdsa (CRYPTO-10)

In [ ]:
from ecdsa import SigningKey, VerifyingKey, SECP256k1, Ed25519

print("ecdsa загружена успешно")
print(f"secp256k1 порядок группы: {SECP256k1.order}")
print(f"secp256k1 размер ключа: {SECP256k1.baselen} байт")

In [ ]:
# Генерация ключей secp256k1 (Bitcoin, Ethereum)
sk_secp = SigningKey.generate(curve=SECP256k1)
vk_secp = sk_secp.get_verifying_key()

print("=== secp256k1 (Bitcoin/Ethereum) ===")
print(f"Приватный ключ (hex): {sk_secp.to_string().hex()}")
print(f"Публичный ключ (hex): {vk_secp.to_string().hex()}")
print(f"Размер приватного ключа: {len(sk_secp.to_string())} байт")
print(f"Размер публичного ключа: {len(vk_secp.to_string())} байт")

# Подпись
message = b"Transfer 1 BTC to Alice"
signature = sk_secp.sign(message)
print(f"\nСообщение: {message.decode()}")
print(f"Подпись: {signature.hex()[:32]}...")
print(f"Размер подписи: {len(signature)} байт")

# Верификация
assert vk_secp.verify(signature, message)
print("Подпись верифицирована!")

In [ ]:
# Генерация ключей Ed25519 (Solana, Polkadot)
sk_ed = SigningKey.generate(curve=Ed25519)
vk_ed = sk_ed.get_verifying_key()

print("=== Ed25519 (Solana/Polkadot) ===")
print(f"Приватный ключ (hex): {sk_ed.to_string().hex()}")
print(f"Публичный ключ (hex): {vk_ed.to_string().hex()}")
print(f"Размер приватного ключа: {len(sk_ed.to_string())} байт")
print(f"Размер публичного ключа: {len(vk_ed.to_string())} байт")

# Подпись
sig_ed = sk_ed.sign(message)
print(f"\nПодпись: {sig_ed.hex()[:32]}...")
print(f"Размер подписи: {len(sig_ed)} байт")

assert vk_ed.verify(sig_ed, message)
print("Подпись верифицирована!")

In [ ]:
# Сравнение производительности
N_ITERS = 100

# secp256k1
start = time.time()
for _ in range(N_ITERS):
    sk = SigningKey.generate(curve=SECP256k1)
secp_gen_time = (time.time() - start) / N_ITERS

sk_bench = SigningKey.generate(curve=SECP256k1)
vk_bench = sk_bench.get_verifying_key()
start = time.time()
for _ in range(N_ITERS):
    sig = sk_bench.sign(b"benchmark")
secp_sign_time = (time.time() - start) / N_ITERS

start = time.time()
for _ in range(N_ITERS):
    vk_bench.verify(sig, b"benchmark")
secp_verify_time = (time.time() - start) / N_ITERS

# Ed25519
start = time.time()
for _ in range(N_ITERS):
    sk = SigningKey.generate(curve=Ed25519)
ed_gen_time = (time.time() - start) / N_ITERS

sk_bench_ed = SigningKey.generate(curve=Ed25519)
vk_bench_ed = sk_bench_ed.get_verifying_key()
start = time.time()
for _ in range(N_ITERS):
    sig_ed = sk_bench_ed.sign(b"benchmark")
ed_sign_time = (time.time() - start) / N_ITERS

start = time.time()
for _ in range(N_ITERS):
    vk_bench_ed.verify(sig_ed, b"benchmark")
ed_verify_time = (time.time() - start) / N_ITERS

print(f"{'Операция':20} {'secp256k1':>12} {'Ed25519':>12} {'Быстрее':>10}")
print("-" * 56)
print(f"{'Генерация ключа':20} {secp_gen_time*1000:>10.2f}мс {ed_gen_time*1000:>10.2f}мс {secp_gen_time/ed_gen_time:>8.1f}x")
print(f"{'Подпись':20} {secp_sign_time*1000:>10.2f}мс {ed_sign_time*1000:>10.2f}мс {secp_sign_time/ed_sign_time:>8.1f}x")
print(f"{'Верификация':20} {secp_verify_time*1000:>10.2f}мс {ed_verify_time*1000:>10.2f}мс {secp_verify_time/ed_verify_time:>8.1f}x")

In [ ]:
# Вычисление публичного ключа из приватного вручную (secp256k1)
# Публичный ключ = d * G, где G - генератор кривой

from ecdsa import SECP256k1, numbertheory
from ecdsa.ellipticcurve import Point

# Параметры secp256k1
curve = SECP256k1.curve
G = SECP256k1.generator
n = SECP256k1.order

print(f"secp256k1 параметры:")
print(f"  a = {curve.a()}")
print(f"  b = {curve.b()}")
print(f"  p = {curve.p()}")
print(f"  G.x = {G.x()}")
print(f"  G.y = {G.y()}")
print(f"  n (порядок) = {n}")

# Генерируем приватный ключ
sk = SigningKey.generate(curve=SECP256k1)
d = int.from_bytes(sk.to_string(), 'big')  # приватный ключ как число

# Ручное вычисление публичного ключа: Q = d * G
Q_manual = d * G  # ecdsa library поддерживает умножение точки на скаляр

# Получаем публичный ключ через API
vk = sk.get_verifying_key()
Q_x = int.from_bytes(vk.to_string()[:32], 'big')
Q_y = int.from_bytes(vk.to_string()[32:], 'big')

print(f"\nПриватный ключ d: {hex(d)[:20]}...")
print(f"Ручное вычисление Q = d*G:")
print(f"  Q.x = {Q_manual.x()}")
print(f"  Q.y = {Q_manual.y()}")
print(f"Через API:")
print(f"  Q.x = {Q_x}")
print(f"  Q.y = {Q_y}")
print(f"\nСовпадают: {Q_manual.x() == Q_x and Q_manual.y() == Q_y}")

---
## Упражнения

### Упражнение 1: Подсчет точек на кривой

Для кривой y^2 = x^3 + 7 (mod p):
1. Подсчитайте количество точек для p = 7, 11, 13, 17, 19, 23
2. Проверьте теорему Хассе: |#E - (p+1)| <= 2*sqrt(p)
3. Для каждого p определите, является ли порядок группы простым числом

In [ ]:
# Упражнение 1: ваш код здесь

def count_curve_points(a: int, b: int, p: int) -> int:
    """Подсчет точек на y^2 = x^3 + ax + b (mod p), включая O."""
    # Ваш код...
    pass

def is_prime(n: int) -> bool:
    """Проверка простоты."""
    if n < 2: return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0: return False
    return True

# for p in [7, 11, 13, 17, 19, 23]:
#     count = count_curve_points(0, 7, p)
#     hasse_bound = 2 * math.sqrt(p)
#     deviation = abs(count - (p + 1))
#     print(f"p={p:>3}: #E={count:>3}, p+1={p+1:>3}, |#E-(p+1)|={deviation:.1f}, 2*sqrt(p)={hasse_bound:.1f}, Хассе: {deviation <= hasse_bound}, Простой: {is_prime(count)}")

### Упражнение 2: Порядок точки

Найдите порядок точки P = (7, 11) на кривой y^2 = x^3 + 7 (mod 23).
Порядок — наименьшее n > 0 такое, что nP = O.

In [ ]:
# Упражнение 2: ваш код здесь

# P = ECPointGF(7, 11, 0, 7, 23)
# for i in range(1, 50):
#     nP = scalar_multiply_gf(i, P)
#     if nP.is_infinity:
#         print(f"Порядок точки P = {i}")
#         break

### Упражнение 3: ECDLP на маленькой кривой

На кривой y^2 = x^3 + 7 (mod 23) с генератором G = (7, 11),
дан публичный ключ Q = (4, 5).
Найдите приватный ключ d такой, что Q = d * G.

**Метод:** перебор (для маленькой кривой).

In [ ]:
# Упражнение 3: ваш код здесь

# G = ECPointGF(7, 11, 0, 7, 23)
# Q_target = ECPointGF(4, 5, 0, 7, 23)
# 
# for d in range(1, 30):
#     if scalar_multiply_gf(d, G) == Q_target:
#         print(f"Найдено: d = {d}, Q = d*G = {scalar_multiply_gf(d, G)}")
#         break

---
## Челлендж: Baby-step Giant-step для ECDLP

Алгоритм Baby-step Giant-step решает ECDLP за O(sqrt(n)) операций вместо O(n).

**Идея:**
1. Выбираем m = ceil(sqrt(n))
2. Baby steps: вычисляем j*P для j = 0, 1, ..., m-1, сохраняем в таблицу
3. Giant steps: вычисляем Q - i*(m*P) для i = 0, 1, ..., m-1
4. Если Q - i*(m*P) == j*P, то d = j + i*m

**Задача:** Реализуйте BSGS и найдите d для Q = d*G на кривой y^2 = x^3 + 7 (mod 23).

In [ ]:
# Челлендж: Baby-step Giant-step

def bsgs(G: ECPointGF, Q: ECPointGF, order: int) -> int:
    """Baby-step Giant-step для ECDLP: найти d такое что Q = d*G."""
    m = math.isqrt(order) + 1
    
    # Baby steps: вычисляем j*G для j = 0..m-1
    baby = {}  # (x, y) -> j
    jG = ECPointGF(None, None, G.a, G.b, G.p)  # O
    for j in range(m):
        if not jG.is_infinity:
            baby[(jG.x, jG.y)] = j
        else:
            baby[('inf', 'inf')] = j
        jG = jG + G
    
    # Giant steps: вычисляем Q - i*(m*G)
    mG = scalar_multiply_gf(m, G)  # m*G
    neg_mG = -mG  # -(m*G)
    
    gamma = Q
    for i in range(m):
        key = ('inf', 'inf') if gamma.is_infinity else (gamma.x, gamma.y)
        if key in baby:
            j = baby[key]
            d = (j + i * m) % order
            return d
        gamma = gamma + neg_mG
    
    return None  # не найдено

# Тест
G_test = ECPointGF(7, 11, 0, 7, 23)

# Сначала найдем порядок точки
order_test = None
for i in range(1, 50):
    if scalar_multiply_gf(i, G_test).is_infinity:
        order_test = i
        break

print(f"G = {G_test}, порядок = {order_test}")

# Тестируем BSGS для разных d
print(f"\n{'d':>4} {'Q = d*G':>12} {'BSGS d':>8} {'OK':>4}")
print("-" * 32)
for d_real in range(1, min(order_test, 20)):
    Q_test = scalar_multiply_gf(d_real, G_test)
    d_found = bsgs(G_test, Q_test, order_test)
    ok = "OK" if d_found == d_real else "FAIL"
    print(f"{d_real:>4} {str(Q_test):>12} {d_found:>8} {ok:>4}")